In [1]:
import pandas as pd
import numpy as np
from market_places_env import MarketPlacesEnv

In [2]:
### Проверка функций step-like

print("Обыграем полный эпизод")

# Инициализируем среду и сбросим
env_ = MarketPlacesEnv()
env_.reset()

# Шаг маркетплейса - повысить ставку
print("Шаг номер 1 - маркетплейсы повысили и понизили ставки")
env_.marketplace_step(1, agent_idx=0)
env_.marketplace_step(2, agent_idx=1)
print(f"Ставки % стали равны {env_.marketplace_rates} \n")

# Шаг дистрибьютора - повысить ставку
print("Шаг номер 2 - дистрибьютор повысил ставку")
env_.distributor_step(2)
print(f"Ставка % стала равна {env_.distributor_rate} \n")

# Шаг выбора фирмы для домохозяйства
print("Шаг номер 3 - выбор работодателя домохозяйствами")
print(f"Текущее распределение по фирмам: {env_.firm_chosen}")
print(f"Пусть все поменяли работодателя")
for ind in range(5):
    env_.household_step(agent_idx=ind, regime='choose_firm', action_=2)
print(f"Стало: {env_.firm_chosen} \n")

# Шаг пересчета зарплат
print("Шаг номер 4 - выплата заработных плат и премий")
print(f"Текущие запасы домохозяйств: {env_.savings_distr}")
for firm_ind in range(2):
    env_.firm_step(agent_idx=firm_ind, regime='wage', action_=2)
print(f"После выплаты зарплат: {env_.savings_distr} \n")

# Шаг производства
print("Шаг номер 5 - производство с учетом нанятых сотрудников")
print(f"Запасы товаров до: {env_.firms_inventory}")
for firm_ind in range(2):
    env_.firm_step(agent_idx=firm_ind, regime='produce', action_=0)
print(f"Запасы товаров после: {env_.firms_inventory} \n")

# Шаг дистрибуции товаров
print("Шаг номер 6 - изменение стратегии дистрибуции")
print(f"Стратегии до: {env_.firms_distribution_attrition}")
print(f"Запасы у дистрибьюторов и МП до: {env_.marketplace_stock + env_.distributor_stock}")
for firm_ind in range(2):
    env_.firm_step(agent_idx=firm_ind, regime='redistribute_inventories', action_=firm_ind % 2)
print(f"Стратегии после: {env_.firms_distribution_attrition} ")
print(f"Запасы у дистрибьюторов и МП после: {env_.marketplace_stock + env_.distributor_stock} \n")

# Шаг с обновлением цен
print("Шаг номер 7 - обновление цен на МП и в оффлайн-магазинах")
print(f"Распределение цен до: {env_.firms_online_prices, env_.firms_offline_prices}")
for firm_ind in range(2):
    for reg in ['prices_online_1', 'prices_online_2', 'price_offline']:
        env_.firm_step(agent_idx=firm_ind, regime=reg, action_=2)
print(f"Распределение цен после: {env_.firms_online_prices, env_.firms_offline_prices} \n")

# Шаг с изменением предпочтений и потреблением
print("Шаг номер 8 - домохозяйства адаптируют предпочтения и потребляют")
print(f"Распределение потребления до: {env_.households_consumption_online}, {env_.households_consumption_offline}")
for house_ind in range(5):
    env_.household_step(regime='redistribute_demand', action_=house_ind, agent_idx=house_ind)
print(f"Распределение потребления после: {env_.households_consumption_online}, {env_.households_consumption_offline}")
print(f"Запасы домохозяйств: {env_.savings_distr}")
print(f"Онлайн и оффлайн-склады: {env_.marketplace_stock}, {env_.distributor_stock} \n")

# Считаем награды
print("Шаг номер 9 - считаем и обновляем награды")
mp_rew_1 = env_.marketplace_reward(agent_idx=0)
mp_rew_2 = env_.marketplace_reward(agent_idx=1)

hh_1 = env_.household_reward(agent_idx=0)
hh_2 = env_. household_reward(agent_idx=1)

firm_1 = env_.firm_reward(agent_idx=0)
firm_2 = env_.firm_reward(agent_idx=1)

distr_rew = env_.distributor_reward()

print(
    f"""
    Награды маркетплейсов: {mp_rew_1}, {mp_rew_2}
    Награда поставщика: {distr_rew}
    Награда домохозяйства: {hh_1}, {hh_2}
    Награда фирм: {firm_1}, {firm_2} \n
    """
)

# Шаг с технологическими шоками
print("Шаг номер 10 - шоки")
print(f"Технологии до: {env_.marketplace_technologies}, {env_.firms_technologies}")
env_.stochastic_step()
print(f"Технологии после: {env_.marketplace_technologies}, {env_.firms_technologies} \n")



# Обновление счетчика эпизода
print(f"Закончился эпизод номер {env_.current_episode}")
env_.current_episode += 1
print(f"Наступил эпизод номер {env_.current_episode}")

Обыграем полный эпизод
Шаг номер 1 - маркетплейсы повысили и понизили ставки
Ставки % стали равны [10, 21] 

Шаг номер 2 - дистрибьютор повысил ставку
Ставка % стала равна 31 

Шаг номер 3 - выбор работодателя домохозяйствами
Текущее распределение по фирмам: [0, 1, 0, 1, 0]
Пусть все поменяли работодателя
Стало: [1, 0, 1, 0, 1] 

Шаг номер 4 - выплата заработных плат и премий
Текущие запасы домохозяйств: [0, 0, 0, 0, 0]
После выплаты зарплат: [33.0, 22.0, 33.0, 22.0, 33.0] 

Шаг номер 5 - производство с учетом нанятых сотрудников
Запасы товаров до: [0, 0]
Запасы товаров после: [4.4, 6.6000000000000005] 

Шаг номер 6 - изменение стратегии дистрибуции
Стратегии до: [[0.3333333333333333, 0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333, 0.3333333333333333]]
Запасы у дистрибьюторов и МП до: [[0, 0], [0, 0], 0, 0]
Стратегии после: [[0.303030303030303, 0.303030303030303, 0.3939393939393939], [0.393939393939394, 0.30303030303030304, 0.30303030303030304]] 
Запас

In [3]:
# Стейты
hh_state = env_.get_household_state(agent_idx=3, regime='redistribute_demand')
f_state = env_.get_firm_state(agent_idx=1, regime='prices_online_1')
mp_state = env_.get_marketplace_state(agent_idx=1)
d_state = env_.get_distributor_state()

print(len(hh_state), hh_state)
print(len(f_state), f_state)
print(len(mp_state), mp_state)
print(len(d_state), d_state)

33 [1, 100, 11.0, 11.0, 11.0, 11.0, 13.200000000000001, 13.200000000000001, 22.0, 33.0, 1, 1.2, 1.521592156160915, 3.9039800888058984, 0.5, 0.0, 0.35757575757575816, 0.0, 0.0, 0.4000000000000001, 0.0, 7.333333333333332, 0.15151515151515152, 0.15151515151515152, 0.24242424242424246, 0.15151515151515152, 0.15151515151515152, 0.15151515151515152, 0.33, 0.5, 0, 31, 1]
52 [1, 100, 0.46264168401572986, 0.19276736833988742, 0.2891510525098311, 0.19276736833988742, 0.2891510525098311, 0.45454545454545453, 0.45454545454545453, 0.30303030303030304, 0.30303030303030304, 0.7272727272727274, 0.45454545454545453, 0.30303030303030304, 0.30303030303030304, 0.45454545454545453, 0.4848484848484848, 11.0, 11.0, 11.0, 11.0, 13.200000000000001, 33.0, 1.2, 1, 1, 1.2, 1.2, 1, 1, 1.2, 1.2, 1, 1, 0, 1, 0, 1, 0, 1.521592156160915, 3.9039800888058984, 10, 21, 0.0, 0.35757575757575816, 0.0, 0.0, 0.5, 0.4000000000000001, 0.0, 31, 3]
19 [1, 100, 11.0, 11.0, 11.0, 11.0, 13.200000000000001, 13.200000000000001, 10, 21